In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../src/')
sys.path.append('../data/')

import player_data_scraper as pds 
import data_cleaning as dc
import scoring as sc

from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

from typing import List

import seaborn as sns
import matplotlib.pyplot as plt

### load data

In [2]:
meta = pd.read_csv('../data/metadata/week02/mls_player_metadata_all.csv')
top_stats = pd.read_csv('../data/top_stats/week02/player_top_stats_corrected.csv')
season = pd.read_csv('../data/season_stats/week02/mls_player_stats_all.csv')

# Features to be used

### Features available at outset

    - team (one hot encode, from season)
    - opponent (one hot encode, from season)
    - position (one hot encode, from meta)
    - salary (value, from meta)
    - home or away match (one hot encode, binary, from season)
    - games played (value, will need some tweaking for match by match calculations)
    - season points (value, from top_stats)
    - 3_week_avg (value, need to calculate)
    - 5_week_avg (value, ned to calculate)
    
### Features to be added with some work and additional datasets

    - 3_week_std (value, need to calculate)
    - 5_week_std (value, need to calculate)
    - days_from_last_match (value, need to calculate)
    - game_time (datetime, one hot encode, for later)
    - weather (temperature, clouds, humidity, etc, for later)

### look at meta df

In [3]:
meta.head()

,ID,name,team,position,salary
0,231969,E. Barco,Atlanta United FC,M,10.5
1,122342,E. Hyndman,Atlanta United FC,M,9.5
2,170797,P. Martinez,Atlanta United FC,M,9.3
3,57769,F. Meza,Atlanta United FC,D,6.9
4,193891,F. Escobar,Atlanta United FC,D,6.9


In [4]:
# make the columns lowercase 
meta.columns = [col.lower() for col in meta.columns]

In [5]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 5 columns):
id          712 non-null int64
name        712 non-null object
team        712 non-null object
position    712 non-null object
salary      712 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 27.9+ KB


In [6]:
meta.head()

,id,name,team,position,salary
0,231969,E. Barco,Atlanta United FC,M,10.5
1,122342,E. Hyndman,Atlanta United FC,M,9.5
2,170797,P. Martinez,Atlanta United FC,M,9.3
3,57769,F. Meza,Atlanta United FC,D,6.9
4,193891,F. Escobar,Atlanta United FC,D,6.9


In [7]:
meta.tail()

,id,name,team,position,salary
707,107672,B. Meredith,Vancouver Whitecaps FC,G,4.5
708,421351,P. Metcalfe,Vancouver Whitecaps FC,M,4.5
709,424258,G. Mukumbilwa,Vancouver Whitecaps FC,D,4.0
710,437697,D. Pecile,Vancouver Whitecaps FC,M,4.0
711,450421,R. Veselinovic,Vancouver Whitecaps FC,D,5.0


In [8]:
# test one hot encoding a column
dc.encode_categories(meta[['position']])

,M,D,G,F
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
707,0.0,0.0,1.0,0.0
708,1.0,0.0,0.0,0.0
709,0.0,1.0,0.0,0.0
710,1.0,0.0,0.0,0.0


In [9]:
# create a dataframe of just features and one hot encoded columns that will be for a features table merged later
meta_features = pd.concat([meta[['id', 'name', 'salary']], 
                           dc.encode_categories(meta[['position']])], axis=1)

In [10]:
meta_features.columns

Index(['id', 'name', 'salary', 'M', 'D', 'G', 'F'], dtype='object')

In [11]:
# rename the columns for the position for the full name for clarity
meta_features.columns = \
    [col for col in ('id', 'name', 'salary', 'midfield', 'defense', 'goalie', 'forward')]

In [13]:
meta_features.head(6)

,id,name,salary,midfield,defense,goalie,forward
0,231969,E. Barco,10.5,1.0,0.0,0.0,0.0
1,122342,E. Hyndman,9.5,1.0,0.0,0.0,0.0
2,170797,P. Martinez,9.3,1.0,0.0,0.0,0.0
3,57769,F. Meza,6.9,0.0,1.0,0.0,0.0
4,193891,F. Escobar,6.9,0.0,1.0,0.0,0.0
5,41705,B. Guzan,6.4,0.0,0.0,1.0,0.0


In [14]:
meta_features.tail(6)

,id,name,salary,midfield,defense,goalie,forward
706,248397,T. Hasal,4.0,0.0,0.0,1.0,0.0
707,107672,B. Meredith,4.5,0.0,0.0,1.0,0.0
708,421351,P. Metcalfe,4.5,1.0,0.0,0.0,0.0
709,424258,G. Mukumbilwa,4.0,0.0,1.0,0.0,0.0
710,437697,D. Pecile,4.0,1.0,0.0,0.0,0.0
711,450421,R. Veselinovic,5.0,0.0,1.0,0.0,0.0


### look at season df

In [15]:
season.head()

,ID,NAME,TEAM,RD,HOME_AWAY,OPPONENT,PTS,MIN,GF,A,...,CRS,BC,CL,BLK,INT,BR,ELG,OGA,SH,WF
0,231969,E. Barco,Atlanta United FC,1,@,Nashville SC,7,90,1,0,...,0,0,0,0,0,3,0,0,2,3
1,231969,E. Barco,Atlanta United FC,2,vs,FC Cincinnati,13,90,1,1,...,0,0,0,0,0,5,0,0,4,5
2,122342,E. Hyndman,Atlanta United FC,1,@,Nashville SC,8,90,1,0,...,0,0,0,0,1,3,0,0,1,0
3,122342,E. Hyndman,Atlanta United FC,2,vs,FC Cincinnati,9,90,1,0,...,0,1,0,1,2,4,0,0,1,0
4,170797,P. Martinez,Atlanta United FC,1,@,Nashville SC,3,81,0,0,...,0,0,1,0,0,3,0,0,1,4


In [16]:
# make the columns all lowercase
season.columns = [col.lower() for col in season.columns]

In [17]:
# for the season df, alter all the numerical columns (which will be the target array) to int from str
for col in season.iloc[:, 7:].columns:
    season[col] = season[col].str.replace('-', '0', regex=False).astype(int)

In [18]:
season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1424 entries, 0 to 1423
Data columns (total 32 columns):
id           1424 non-null int64
name         1424 non-null object
team         1424 non-null object
rd           1424 non-null int64
home_away    1424 non-null object
opponent     1424 non-null object
pts          1424 non-null int64
min          1424 non-null int64
gf           1424 non-null int64
a            1424 non-null int64
cs           1424 non-null int64
ps           1424 non-null int64
pe           1424 non-null int64
pm           1424 non-null int64
ga           1424 non-null int64
sv           1424 non-null int64
y            1424 non-null int64
r            1424 non-null int64
og           1424 non-null int64
t            1424 non-null int64
p            1424 non-null int64
kp           1424 non-null int64
crs          1424 non-null int64
bc           1424 non-null int64
cl           1424 non-null int64
blk          1424 non-null int64
int          1424 non-null int6

In [20]:
# these columns have feature data. everything from these columns on are target data
season.columns[:6]

Index(['id', 'name', 'team', 'rd', 'home_away', 'opponent'], dtype='object')

In [21]:
# make a season features df with one hot encoded columsn for the team the player is on, the opponent they played
# that week, and weather the match was home or away. also, rename the columns '@' as away and 'vs' as home.

# TODO - need too attach 'opp_' to the opponent teams in the columns to differentiate the player's team from 
#the opponent's team

season_features = pd.concat([season[['id', 'name', 'rd']],
                             dc.encode_categories(season[['home_away', 'team', 'opponent']])], 
                             axis = 1).rename(columns={'@': 'away', 'vs': 'home'})

In [22]:
season_features.head(6)

,id,name,rd,away,home,Atlanta United FC,Chicago Fire FC,FC Cincinnati,Columbus Crew SC,D.C. United,...,Real Salt Lake,San Jose Earthquakes,Orlando City SC,Philadelphia Union,LA Galaxy,Sporting Kansas City,Houston Dynamo,Vancouver Whitecaps FC,Portland Timbers,Minnesota United FC
0,231969,E. Barco,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,231969,E. Barco,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,122342,E. Hyndman,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,122342,E. Hyndman,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,170797,P. Martinez,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,170797,P. Martinez,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
season_features.tail(6)

,id,name,rd,away,home,Atlanta United FC,Chicago Fire FC,FC Cincinnati,Columbus Crew SC,D.C. United,...,Real Salt Lake,San Jose Earthquakes,Orlando City SC,Philadelphia Union,LA Galaxy,Sporting Kansas City,Houston Dynamo,Vancouver Whitecaps FC,Portland Timbers,Minnesota United FC
1418,437697,D. Pecile,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1419,437697,D. Pecile,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1420,450421,R. Veselinovic,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1421,450421,R. Veselinovic,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1422,110596,M. Ibarra,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1423,110596,M. Ibarra,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# there are 26 columns for each player that determine their fantasy points for each game. They can be found
# in the season stats csv, starting with index 6, 'pts', and running through the end of the 
# the column.

# TODO - the 'pts' column is actually the summation of the fantasy points earned from the other columns. It 
# can be used to confirm the scoring functions are working properly
season.columns[6:]

Index(['pts', 'min', 'gf', 'a', 'cs', 'ps', 'pe', 'pm', 'ga', 'sv', 'y', 'r',
       'og', 't', 'p', 'kp', 'crs', 'bc', 'cl', 'blk', 'int', 'br', 'elg',
       'oga', 'sh', 'wf'],
      dtype='object')

In [30]:
# there are 25 columns for each player that determine their fantasy points for each game. They can be found
# in the season stats csv, starting with index 7, 'MIN', and running through the end of the 
# the column.
# these will make up the array for the targets

# todo - need to determine if 'pts' columns is needed. perhaps saved separately to test scoring functions

season_target = pd.concat([season[['id', 'name', 'rd']], season[season.columns[6:]]], axis = 1)

In [31]:
season_target.head(6)

,id,name,rd,pts,min,gf,a,cs,ps,pe,...,crs,bc,cl,blk,int,br,elg,oga,sh,wf
0,231969,E. Barco,1,7,90,1,0,0,0,0,...,0,0,0,0,0,3,0,0,2,3
1,231969,E. Barco,2,13,90,1,1,0,0,0,...,0,0,0,0,0,5,0,0,4,5
2,122342,E. Hyndman,1,8,90,1,0,0,0,0,...,0,0,0,0,1,3,0,0,1,0
3,122342,E. Hyndman,2,9,90,1,0,0,0,0,...,0,1,0,1,2,4,0,0,1,0
4,170797,P. Martinez,1,3,81,0,0,0,0,0,...,0,0,1,0,0,3,0,0,1,4
5,170797,P. Martinez,2,11,67,0,2,0,0,0,...,0,2,0,0,0,2,0,0,2,5


In [32]:
season_target.tail(6)

,id,name,rd,pts,min,gf,a,cs,ps,pe,...,crs,bc,cl,blk,int,br,elg,oga,sh,wf
1418,437697,D. Pecile,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1419,437697,D. Pecile,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1420,450421,R. Veselinovic,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1421,450421,R. Veselinovic,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1422,110596,M. Ibarra,1,3,73,0,0,0,0,0,...,0,0,0,0,1,2,0,0,6,0
1423,110596,M. Ibarra,2,1,59,0,0,0,0,0,...,0,0,1,0,2,2,0,0,0,0


### this function wil go through a full season of stats for the and provide the ytd points for a player based on the game week

In [33]:
def calculate_ytd_fantasy_points(df: pd.DataFrame, game_week: int) -> pd.DataFrame:
    '''Calculate the year-to-date fantasy points for each player up to and including the current round.
    To used specifically with the season stats dataframe for MLS soccer.
    '''
    # TODO - assert statements to confirm the dataframe has the appropriate columns
    return df[df['rd'] <= game_week][['name', 'id', 'rd', 'pts']].groupby(['id', 'name']).sum()

In [34]:
rd_1_points = calculate_ytd_fantasy_points(season[season['rd'] == 1], 1)
rd_2_points = calculate_ytd_fantasy_points(season[season['rd'] == 2], 2)

In [35]:
rd_1_points.columns

Index(['rd', 'pts'], dtype='object')

In [36]:
rd_1_points

,,rd,pts
id,name,,
13356,R. Fanni,1,5
15517,K. Beckerman,1,0
18222,G. Rossi,1,1
18529,R. Ziegler,1,8
18770,B. Wright-Phillips,1,0
...,...,...,...
501094,T. Beason,1,0
501277,M. Berry,1,0
501651,N. Burgess,1,0


In [37]:
rd_2_points

,,rd,pts
id,name,,
13356,R. Fanni,2,4
15517,K. Beckerman,2,0
18222,G. Rossi,2,0
18529,R. Ziegler,2,4
18770,B. Wright-Phillips,2,0
...,...,...,...
501094,T. Beason,2,0
501277,M. Berry,2,0
501651,N. Burgess,2,0


### look at top_stats df

In [38]:
top_stats.head()

,id,name,team,games_played,avg_fantasy_pts,total_fantasy_pts,last_wk_fantasy_pts,3_wk_avg,5_wk_avg,high_score,low_score,owned_by,$/point,rd_2_rank,season_rank
0,231969,E. Barco,Atlanta United FC,2,10.0,20,13,9.00,7.0,13,7,23.61,$525K,3,5
1,122342,E. Hyndman,Atlanta United FC,2,8.5,17,9,6.33,5.4,9,8,3.96,$559K,21,17
2,170797,P. Martinez,Atlanta United FC,2,7.0,14,11,5.67,7.0,11,3,20.57,$665K,7,32
3,57769,F. Meza,Atlanta United FC,2,5.5,11,7,5.50,5.5,7,4,3.07,$628K,61,88
4,193891,F. Escobar,Atlanta United FC,1,5.0,5,DNP,3.33,4.6,5,5,5.81,""" $1380K""",0,112


In [39]:
#rename columns '$/point' as 'price_per_point'
top_stats = top_stats.rename(columns={'$/point': 'price_per_point'})

In [40]:
top_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 15 columns):
id                     712 non-null int64
name                   712 non-null object
team                   712 non-null object
games_played           712 non-null int64
avg_fantasy_pts        712 non-null float64
total_fantasy_pts      712 non-null int64
last_wk_fantasy_pts    712 non-null object
3_wk_avg               712 non-null float64
5_wk_avg               712 non-null float64
high_score             712 non-null int64
low_score              712 non-null int64
owned_by               712 non-null float64
price_per_point        712 non-null object
rd_2_rank              712 non-null int64
season_rank            712 non-null int64
dtypes: float64(4), int64(7), object(4)
memory usage: 83.6+ KB


In [41]:
top_stats['last_wk_fantasy_pts'].value_counts(), top_stats['last_wk_fantasy_pts'].count()

(-      311
 1       80
 3       54
 2       52
 DNP     45
 4       45
 5       27
 9       20
 8       20
 6       16
 7       13
 10      11
 0        8
 11       3
 13       3
 17       2
 -2       1
 12       1
 Name: last_wk_fantasy_pts, dtype: int64, 712)

In [42]:
# steps to clean up data for 'last_wk_fantasy_stats'
top_stats['last_wk_fantasy_pts'] = top_stats['last_wk_fantasy_pts'].str.replace('DNP', '0', regex=False)
top_stats['last_wk_fantasy_pts'] = pd.to_numeric(top_stats['last_wk_fantasy_pts'], errors='coerce')
top_stats['last_wk_fantasy_pts'].fillna(0, inplace=True)

In [44]:
# check that 'DNP' and '-' are now 0.
top_stats['last_wk_fantasy_pts'].value_counts(), top_stats['last_wk_fantasy_pts'].count()

( 0.0     364
  1.0      80
  3.0      54
  2.0      52
  4.0      45
  5.0      27
  8.0      20
  9.0      20
  6.0      16
  7.0      13
  10.0     11
  11.0      3
  13.0      3
  17.0      2
  12.0      1
 -2.0       1
 Name: last_wk_fantasy_pts, dtype: int64, 712)

In [45]:
# steps to clean up 'price_per_point' so the remaining values are floats

# TODO - if time, pursue regex here instead of a step for each unwanted character

top_stats['price_per_point'] = top_stats['price_per_point'].str.replace(' ', '')
top_stats['price_per_point'] = top_stats['price_per_point'].str.replace("'", "")
top_stats['price_per_point'] = top_stats['price_per_point'].str.replace('"', '')
top_stats['price_per_point'] = top_stats['price_per_point'].str.replace('$', '')
top_stats['price_per_point'] = top_stats['price_per_point'].str.replace('K', '')
top_stats['price_per_point'] = top_stats['price_per_point'].str.replace(',', '')
top_stats['price_per_point'] = top_stats['price_per_point'].astype(float)

In [46]:
top_stats['price_per_point'].describe()

count      712.000000
mean      3050.196629
std       2174.929154
min      -2650.000000
25%        844.500000
50%       4000.000000
75%       4500.000000
max      12000.000000
Name: price_per_point, dtype: float64

In [47]:
top_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 15 columns):
id                     712 non-null int64
name                   712 non-null object
team                   712 non-null object
games_played           712 non-null int64
avg_fantasy_pts        712 non-null float64
total_fantasy_pts      712 non-null int64
last_wk_fantasy_pts    712 non-null float64
3_wk_avg               712 non-null float64
5_wk_avg               712 non-null float64
high_score             712 non-null int64
low_score              712 non-null int64
owned_by               712 non-null float64
price_per_point        712 non-null float64
rd_2_rank              712 non-null int64
season_rank            712 non-null int64
dtypes: float64(6), int64(7), object(2)
memory usage: 83.6+ KB


In [48]:
top_stats.columns

Index(['id', 'name', 'team', 'games_played', 'avg_fantasy_pts',
       'total_fantasy_pts', 'last_wk_fantasy_pts', '3_wk_avg', '5_wk_avg',
       'high_score', 'low_score', 'owned_by', 'price_per_point', 'rd_2_rank',
       'season_rank'],
      dtype='object')

In [50]:
# need to use the stats from the season df to calculate for each game week 'games_played', 'avg_fantasy_points', 
# 'total_fantasy_points', 'last_wk_fantasy_points', '3_wk_avg', '5_wk_avg', 'high_score', 'low_score', 'owned_by',
# and 'price_per_point'. 'rd_2_rank' (effectively the rank the last game of the points earned for their position)
# and 'season_rank' (the rank by position) could be valuable but I don't trust the stats from mls

top_stats_features = top_stats[['id', 'name', 'games_played', 'avg_fantasy_pts', 'total_fantasy_pts',
                                'last_wk_fantasy_pts', '3_wk_avg', '5_wk_avg', 'high_score', 'low_score', 
                                'owned_by', 'price_per_point']]

In [51]:
top_stats_features.head()

,id,name,games_played,avg_fantasy_pts,total_fantasy_pts,last_wk_fantasy_pts,3_wk_avg,5_wk_avg,high_score,low_score,owned_by,price_per_point
0,231969,E. Barco,2,10.0,20,13.0,9.00,7.0,13,7,23.61,525.0
1,122342,E. Hyndman,2,8.5,17,9.0,6.33,5.4,9,8,3.96,559.0
2,170797,P. Martinez,2,7.0,14,11.0,5.67,7.0,11,3,20.57,665.0
3,57769,F. Meza,2,5.5,11,7.0,5.50,5.5,7,4,3.07,628.0
4,193891,F. Escobar,1,5.0,5,0.0,3.33,4.6,5,5,5.81,1380.0


In [52]:
top_stats_features.tail()

,id,name,games_played,avg_fantasy_pts,total_fantasy_pts,last_wk_fantasy_pts,3_wk_avg,5_wk_avg,high_score,low_score,owned_by,price_per_point
707,107672,B. Meredith,0,0.0,0,0.0,1.0,0.6,0,0,0.11,4500.0
708,421351,P. Metcalfe,0,0.0,0,0.0,0.0,0.0,0,0,0.02,4500.0
709,424258,G. Mukumbilwa,0,0.0,0,0.0,1.0,1.0,0,0,0.47,4000.0
710,437697,D. Pecile,0,0.0,0,0.0,0.0,0.0,0,0,0.28,4000.0
711,450421,R. Veselinovic,0,0.0,0,0.0,0.0,0.0,0,0,0.04,5000.0


### now combine the three dfs to create a table of all features and targets, which will separated later for modeling

In [58]:
# check filtering season_features by 'rd'
season_features[season_features['rd'] == 2]

,id,name,rd,away,home,Atlanta United FC,Chicago Fire FC,FC Cincinnati,Columbus Crew SC,D.C. United,...,Real Salt Lake,San Jose Earthquakes,Orlando City SC,Philadelphia Union,LA Galaxy,Sporting Kansas City,Houston Dynamo,Vancouver Whitecaps FC,Portland Timbers,Minnesota United FC
1,231969,E. Barco,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,122342,E. Hyndman,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,170797,P. Martinez,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,57769,F. Meza,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,193891,F. Escobar,2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,421351,P. Metcalfe,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1417,424258,G. Mukumbilwa,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1419,437697,D. Pecile,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1421,450421,R. Veselinovic,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [59]:
top_stats_features.tail()

,id,name,games_played,avg_fantasy_pts,total_fantasy_pts,last_wk_fantasy_pts,3_wk_avg,5_wk_avg,high_score,low_score,owned_by,price_per_point
707,107672,B. Meredith,0,0.0,0,0.0,1.0,0.6,0,0,0.11,4500.0
708,421351,P. Metcalfe,0,0.0,0,0.0,0.0,0.0,0,0,0.02,4500.0
709,424258,G. Mukumbilwa,0,0.0,0,0.0,1.0,1.0,0,0,0.47,4000.0
710,437697,D. Pecile,0,0.0,0,0.0,0.0,0.0,0,0,0.28,4000.0
711,450421,R. Veselinovic,0,0.0,0,0.0,0.0,0.0,0,0,0.04,5000.0


In [62]:
# check the shape of the three dfs with features to make sure merged df has all the columns

meta_features.shape, season_features[season_features['rd'] == 2].shape, top_stats_features.shape

((712, 7), (712, 57), (712, 12))

In [66]:
# subtracting 2 columns because we are joining on 'id' for each merge in the chain
7 + 57 + 12 - 2

74

In [67]:
# Merge meta_features, season_features for round 2, and top_stats_features into one features db, joined on
# player id

merged = pd.merge(meta_features, 
                  top_stats_features, 
                  how='outer', 
                  left_on='id', 
                  right_on='id', 
                  suffixes=('_meta', '_top')).merge(season_features[season_features['rd'] == 2],
                                                    how='outer',
                                                    left_on='id', 
                                                    right_on='id')

features_merged = pd.merge(meta_features, 
                  top_stats_features, 
                  how='outer', 
                  left_on='id', 
                  right_on='id', 
                  suffixes=('_meta', '_top')).merge(season,
                                                    how='outer',
                                                    left_on='id',
                                                    right_on='id')

In [68]:
merged

,id,name_meta,salary,midfield,defense,goalie,forward,name_top,games_played,avg_fantasy_pts,...,Real Salt Lake,San Jose Earthquakes,Orlando City SC,Philadelphia Union,LA Galaxy,Sporting Kansas City,Houston Dynamo,Vancouver Whitecaps FC,Portland Timbers,Minnesota United FC
0,231969,E. Barco,10.5,1.0,0.0,0.0,0.0,E. Barco,2,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,122342,E. Hyndman,9.5,1.0,0.0,0.0,0.0,E. Hyndman,2,8.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,170797,P. Martinez,9.3,1.0,0.0,0.0,0.0,P. Martinez,2,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57769,F. Meza,6.9,0.0,1.0,0.0,0.0,F. Meza,2,5.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,193891,F. Escobar,6.9,0.0,1.0,0.0,0.0,F. Escobar,1,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,107672,B. Meredith,4.5,0.0,0.0,1.0,0.0,B. Meredith,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
708,421351,P. Metcalfe,4.5,1.0,0.0,0.0,0.0,P. Metcalfe,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
709,424258,G. Mukumbilwa,4.0,0.0,1.0,0.0,0.0,G. Mukumbilwa,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
710,437697,D. Pecile,4.0,1.0,0.0,0.0,0.0,D. Pecile,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [69]:
features_merged

,id,name_meta,salary,midfield,defense,goalie,forward,name_top,games_played,avg_fantasy_pts,...,crs,bc,cl,blk,int,br,elg,oga,sh,wf
0,231969,E. Barco,10.5,1.0,0.0,0.0,0.0,E. Barco,2,10.0,...,0,0,0,0,0,3,0,0,2,3
1,231969,E. Barco,10.5,1.0,0.0,0.0,0.0,E. Barco,2,10.0,...,0,0,0,0,0,5,0,0,4,5
2,122342,E. Hyndman,9.5,1.0,0.0,0.0,0.0,E. Hyndman,2,8.5,...,0,0,0,0,1,3,0,0,1,0
3,122342,E. Hyndman,9.5,1.0,0.0,0.0,0.0,E. Hyndman,2,8.5,...,0,1,0,1,2,4,0,0,1,0
4,170797,P. Martinez,9.3,1.0,0.0,0.0,0.0,P. Martinez,2,7.0,...,0,0,1,0,0,3,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,424258,G. Mukumbilwa,4.0,0.0,1.0,0.0,0.0,G. Mukumbilwa,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1420,437697,D. Pecile,4.0,1.0,0.0,0.0,0.0,D. Pecile,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1421,437697,D. Pecile,4.0,1.0,0.0,0.0,0.0,D. Pecile,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1422,450421,R. Veselinovic,5.0,0.0,1.0,0.0,0.0,R. Veselinovic,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
features_merged.columns

Index(['id', 'name_meta', 'salary', 'midfield', 'defense', 'goalie', 'forward',
       'name_top', 'games_played', 'avg_fantasy_pts', 'total_fantasy_pts',
       'last_wk_fantasy_pts', '3_wk_avg', '5_wk_avg', 'high_score',
       'low_score', 'owned_by', 'price_per_point', 'name', 'team', 'rd',
       'home_away', 'opponent', 'pts', 'min', 'gf', 'a', 'cs', 'ps', 'pe',
       'pm', 'ga', 'sv', 'y', 'r', 'og', 't', 'p', 'kp', 'crs', 'bc', 'cl',
       'blk', 'int', 'br', 'elg', 'oga', 'sh', 'wf'],
      dtype='object')

In [71]:
season_features.columns

Index(['id', 'name', 'rd', 'away', 'home', 'Atlanta United FC',
       'Chicago Fire FC', 'FC Cincinnati', 'Columbus Crew SC', 'D.C. United',
       'Inter Miami CF', 'Montreal Impact', 'New England Revolution',
       'New York City FC', 'New York Red Bulls', 'Orlando City SC',
       'Philadelphia Union', 'Toronto FC', 'Colorado Rapids', 'FC Dallas',
       'Houston Dynamo', 'LA Galaxy', 'Los Angeles FC', 'Minnesota United FC',
       'Nashville SC', 'Portland Timbers', 'Real Salt Lake',
       'San Jose Earthquakes', 'Seattle Sounders FC', 'Sporting Kansas City',
       'Vancouver Whitecaps FC', 'Nashville SC', 'FC Cincinnati',
       'Seattle Sounders FC', 'New England Revolution', 'New York Red Bulls',
       'Atlanta United FC', 'New York City FC', 'Colorado Rapids',
       'Inter Miami CF', 'Los Angeles FC', 'D.C. United', 'FC Dallas',
       'Montreal Impact', 'Chicago Fire FC', 'Columbus Crew SC', 'Toronto FC',
       'Real Salt Lake', 'San Jose Earthquakes', 'Orlando City SC'

In [72]:
season.columns

Index(['id', 'name', 'team', 'rd', 'home_away', 'opponent', 'pts', 'min', 'gf',
       'a', 'cs', 'ps', 'pe', 'pm', 'ga', 'sv', 'y', 'r', 'og', 't', 'p', 'kp',
       'crs', 'bc', 'cl', 'blk', 'int', 'br', 'elg', 'oga', 'sh', 'wf'],
      dtype='object')

In [ ]:
merged.columns[15:]

### as of 6/17/20, everything below here was used to create the encode_categories() function, which is now in pulled in from the data_cleaning file

In [ ]:
def encode_categories(df: pd.DataFrame) -> pd.DataFrame:
    '''Will take dataframe and create a one-hot data frame with columns for all unique values in each column
    of the dataframe.
    '''
    df_cols = [df[col].unique() for col in df.columns]
    
    enc = OneHotEncoder(categories=df_cols, handle_unknown='ignore')
    ohe_cols = [elem for cat in enc.categories for elem in cat]
    
    encoded_df = enc.fit_transform(df.values).toarray()
    encoded_df = pd.DataFrame(encoded_df, columns=ohe_cols)
    
    return encoded_df